In [ ]:
#| default_exp data.database.sqlalchemy

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import pandas as pd 

In [ ]:
from sqlalchemy import MetaData, create_engine, asc, desc, and_, or_, not_
from sqlalchemy.schema import *
from sqlalchemy.sql import func as F, Selectable, select
from sqlalchemy.dialects import registry
from sqlalchemy.engine.row import Row
from sqlalchemy.orm import Session

In [ ]:
#| export
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials


In [ ]:
#| hide
from google.cloud import bigquery
from dotenv import load_dotenv


In [ ]:

load_dotenv("../.env")

In [ ]:
os.environ

In [ ]:
# os.getenv('GC_QUOTE_API_CREDENTIALS')
# os.getenv('HTTP_PROXY')
# os.getenv('HTTPS_PROXY')

In [ ]:
# config = dotenv_values("../.env")
# key = config.get('GC_QUOTE_API_CREDENTIALS')
key = os.getenv('GC_QUOTE_API_CREDENTIALS')

In [ ]:
credential = service_account.Credentials.from_service_account_file("../" + key)


In [ ]:

#os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
#os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
#os.environ['http_proxy'] = "http://127.0.0.1:20171"
#os.environ['https_proxy'] = "http://127.0.0.1:20171"



In [ ]:
registry.register('bigquery', 'sqlalchemy_bigquery', 'BigQueryDialect')
engine = create_engine('bigquery://quote-api-365206',
                       credentials_path='../' + key,)


In [ ]:
metadata = MetaData()
def get_table(project_name: str, dataset_name: str, table_name: str)-> Table:
    table = Table(f'{project_name}.{dataset_name}.{table_name}', metadata, autoload_with=engine)
    return table

In [ ]:
# table = get_table('quote-api-365206', 'report', 'ride_dispatch_v')

In [ ]:
# query = select(table).limit(20).where(table.c.ride_stat == 'Accepted').order_by(asc(table.c.ride_id))
# df = pd.read_sql(query, engine)
# df


In [ ]:
# table = get_table('elife-data-warehouse-prod', 'ods', 'ride_ride')

In [ ]:
# table

In [ ]:
# query = select(table).where(table.c.airport_code3 == 'JFK').limit(20)

In [ ]:
# df = pd.read_sql(query, engine)
# df


# find all tables in the database

In [ ]:
ride_trip_t = get_table('elife-data-warehouse-prod','ods', 'ride_trip').alias()
ride_dispatch_t = get_table('elife-data-warehouse-prod','ods', 'ride_dispatch').alias()
ride_enum_t = get_table('elife-data-warehouse-prod','ods', 'ride_enum').alias()
ride_ride_t = get_table('elife-data-warehouse-prod','ods', 'ride_ride').alias()
ride_partner_tran_t = get_table('elife-data-warehouse-prod','ods', 'ride_partner_tran').alias()
ride_partner_t = get_table('elife-data-warehouse-prod','ods', 'ride_partner').alias()
dim_place_t = get_table('elife-data-warehouse-prod','dim', 'dim_place').alias()
ride_vehicle_class_t = get_table('elife-data-warehouse-prod','ods', 'ride_vehicle_class').alias()

In [ ]:
trip_type_id_t = select(ride_dispatch_t.c.ride_id, ride_trip_t.c.trip_type.label('trip_type_id'))
trip_type_id_t = trip_type_id_t.select_from(ride_dispatch_t
                                      .join(ride_trip_t, ride_dispatch_t.c.ride_id == ride_trip_t.c.ride_id))
trip_type_id_t = trip_type_id_t.alias()
                  

In [ ]:
# trip_type_id_t = trip_type_id_t.limit(10)
# df = pd.read_sql(trip_type_id_t, engine)
# df

In [ ]:
trip_type_t = select(trip_type_id_t.c.ride_id, trip_type_id_t.c.trip_type_id, ride_enum_t.c.name.label('trip_type'))
trip_type_t = trip_type_t.select_from(trip_type_id_t
                                      .join(ride_enum_t, trip_type_id_t.c.trip_type_id == ride_enum_t.c.id, isouter=True))
trip_type_t = trip_type_t.alias()


In [ ]:
ride_status_t = select(ride_ride_t.c.id.label('ride_id'), ride_ride_t.c.stat.label('ride_status_id'), ride_enum_t.c.name.label('ride_status'))
ride_status_t = ride_status_t.select_from(ride_ride_t
                                          .join(ride_enum_t, ride_ride_t.c.stat == ride_enum_t.c.id, isouter=True))
ride_status_t = ride_status_t.alias()
# ride_status_t = ride_status_t.limit(10)
# df = pd.read_sql(ride_status_t, engine)
# df

In [ ]:
partner_id_t = select(ride_ride_t.c.id.label('ride_id'), ride_partner_tran_t.c.partner_id.label('partner_id'))
partner_id_t = partner_id_t.select_from(ride_ride_t
                                        .join(ride_partner_tran_t, ride_ride_t.c.partner_tran_id == ride_partner_tran_t.c.id, isouter=True))
partner_id_t = partner_id_t.alias()
#partner_id_t = partner_id_t.limit(10)
#df = pd.read_sql(partner_id_t, engine)
#df

In [ ]:
partner_t = select(partner_id_t.c.ride_id, partner_id_t.c.partner_id, ride_partner_t.c.name.label('partner'))
partner_t = partner_t.select_from(partner_id_t
                                  .join(ride_partner_t, partner_id_t.c.partner_id == ride_partner_t.c.id, isouter=True))
partner_t = partner_t.alias()
# partner_t = partner_t.limit(10)
# df = pd.read_sql(partner_t,engine)
# df

In [ ]:
from_place_t = select(ride_ride_t.c.id.label('ride_id'), 
                      ride_ride_t.c.from_place_id.label('start_place_id'), 
                      dim_place_t.c.name.label('start_place'), 
                      dim_place_t.c.lng.label('lng'), 
                      dim_place_t.c.lat.label('ltt'))
from_place_t = from_place_t.select_from(ride_ride_t
                                        .join(dim_place_t, ride_ride_t.c.from_place_id == dim_place_t.c.id, isouter=True))
from_place_t = from_place_t.alias()
#from_place_t = from_place_t.limit(10)
#df = pd.read_sql(from_place_t, engine)
#df

In [ ]:
to_place_t = select(ride_ride_t.c.id.label('ride_id'), 
                      ride_ride_t.c.to_place_id.label('end_place_id'), 
                      dim_place_t.c.name.label('end_place'), 
                      dim_place_t.c.lng.label('lng'), 
                      dim_place_t.c.lat.label('ltt'))
to_place_t = to_place_t.select_from(ride_ride_t
                                        .join(dim_place_t, ride_ride_t.c.to_place_id == dim_place_t.c.id, isouter=True))
to_place_t = to_place_t.alias()
#to_place_t = to_place_t.limit(10)
#df = pd.read_sql(to_place_t, engine)
#df

In [ ]:
vehicle_class_t = select(ride_ride_t.c.id.label('ride_id'),
                         ride_ride_t.c.vehicle_class_id.label('vehicle_class_id'), 
                         ride_vehicle_class_t.c.name.label('vehicle_class'))
vehicle_class_t = vehicle_class_t.select_from(ride_ride_t
                                              .join(ride_vehicle_class_t, ride_ride_t.c.vehicle_class_id == ride_vehicle_class_t.c.id, isouter=True))
vehicle_class_t = vehicle_class_t.alias()
#vehicle_class_t = vehicle_class_t.limit(10)
#df = pd.read_sql(vehicle_class_t, engine)
#df

In [ ]:
price_training_t = select(ride_ride_t.c.id.label('ride_id'),
                          ride_dispatch_t.c.id.label('dispatch_id'),
                          trip_type_t.c.trip_type_id,
                          trip_type_t.c.trip_type,
                          ride_ride_t.c.trip_count,
                          ride_dispatch_t.c.trip_no,
                          ride_status_t.c.ride_status_id,
                          ride_status_t.c.ride_status,
                          partner_t.c.partner_id,
                          partner_t.c.partner,
                          ride_ride_t.c.from_time_str,
                          ride_ride_t.c.from_timezone_str,
                          ride_ride_t.c.to_time_str,
                          ride_ride_t.c.to_timezone_str,
                          from_place_t.c.start_place_id,
                          from_place_t.c.start_place,
                          from_place_t.c.lng.label('start_lng'),
                          from_place_t.c.ltt.label('start_ltt'),
                          to_place_t.c.end_place_id,
                          to_place_t.c.end_place,
                          to_place_t.c.lng.label('end_lng'),
                          to_place_t.c.ltt.label('end_ltt'),
                          ride_ride_t.c.distance,
                          ride_ride_t.c.duration,
                          vehicle_class_t.c.vehicle_class_id,
                          vehicle_class_t.c.vehicle_class,
                          ride_ride_t.c.passenger_count,
                          ride_ride_t.c.luggage_count,
                          ride_ride_t.c.children_count,
                          ride_ride_t.c.infant_count,
                          ride_dispatch_t.c.amount.label('dispatch_amount'),
                          ride_dispatch_t.c.currency.label('dispatch_currency')).where(
                              and_(
                                  ride_ride_t.c.from_time_str > '2024-01-01',
                                  ride_dispatch_t.c.currency == 'USD')
                          )


In [ ]:

price_training_t = price_training_t.select_from(ride_ride_t
                                                .join(ride_dispatch_t, ride_ride_t.c.id == ride_dispatch_t.c.ride_id)
                                                .join(trip_type_t, ride_ride_t.c.id == trip_type_t.c.ride_id, isouter=True)
                                                .join(ride_status_t, ride_ride_t.c.id == ride_status_t.c.ride_id, isouter=True)
                                                .join(partner_t, ride_ride_t.c.id == partner_t.c.ride_id, isouter=True)
                                                .join(from_place_t, ride_ride_t.c.id == from_place_t.c.ride_id, isouter=True)
                                                .join(to_place_t, ride_ride_t.c.id == to_place_t.c.ride_id, isouter=True)
                                                .join(vehicle_class_t, ride_ride_t.c.id == vehicle_class_t.c.ride_id, isouter=True)
                                                )
                                                                                                                                    
price_training_t = price_training_t.where(ride_ride_t.c.from_time_str > '2024-01-01')

In [ ]:
price_training_t = price_training_t.limit(200)

In [ ]:
df = pd.read_sql(price_training_t, engine)
df

In [ ]:
df_training = df.loc[:,['ride_id', 'trip_type', 'trip_no', 'trip_count', 'ride_status', 'partner', 
                'from_time_str', 'from_timezone_str', 'to_time_str', 'to_timezone_str', 
                'start_place', 'start_ltt', 'start_lng', 'end_place', 'end_ltt', 'end_lng', 
                'distance', 'duration', 'vehicle_class', 
                'passenger_count', 'luggage_count', 'children_count', 'infant_count', 
                'dispatch_amount', 'dispatch_currency']]
df_training['price_per_km'] = df_training['dispatch_amount'] / df_training['distance'] * 100.0
df_training

In [ ]:
#| export
QUERY_PRICING_TRAINING_FILTER = """
        WITH
        TripTypeID AS (
            SELECT
                t1.ride_id,
                CASE
                    WHEN t2.trip_no = t1.trip_no THEN t2.trip_type
                    ELSE NULL
                END AS trip_type_id,
            FROM
                elife-data-warehouse-prod.ods.ride_dispatch AS t1
                JOIN elife-data-warehouse-prod.ods.ride_trip AS t2 ON t1.ride_id = t2.ride_id
        ),
        TripType AS (
            SELECT
                t1.ride_id,
                t1.trip_type_id,
                t2.name AS trip_type,
            FROM
                TripTypeID AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_enum AS t2 ON t1.trip_type_id = t2.id
        ),
        RideStatus AS (
            SELECT
                t1.id AS ride_id,
                t1.stat AS ride_status_id,
                t2.name AS ride_status,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_enum AS t2 ON t1.stat = t2.id
        ),
        PartnerID AS (
            SELECT
                t1.id AS ride_id,
                CASE
                    WHEN t2.id = t1.partner_tran_id THEN t2.partner_id
                    ELSE NULL
                END AS partner_id,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_partner_tran AS t2 ON t1.partner_tran_id = t2.id
        ),
        Partner AS (
            SELECT
                t1.ride_id,
                t1.partner_id,
                t2.name AS partner,
            FROM
                PartnerID AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_partner AS t2 ON t1.partner_id = t2.id
        ),
        FromPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.from_place_id as start_place_id,
                t2.name AS start_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.dim.dim_place AS t2 ON t1.from_place_id = t2.id
        ),
        ToPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.to_place_id as end_place_id,
                t2.name AS end_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.dim.dim_place AS t2 ON t1.to_place_id = t2.id
        ),
        VehicleClass AS (
            SELECT
                t1.id AS ride_id,
                t1.vehicle_class_id as vehicle_class_id,
                t2.name AS vehicle_class,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_vehicle_class AS t2 ON t1.vehicle_class_id = t2.id
        ),
        PricingTraining AS (SELECT t0.id       AS ride_id,
                                   t1.id       AS dispatch_id,
                                   t2.trip_type_id,
                                   t2.trip_type,
                                   t1.trip_no,
                                   t3.ride_status_id,
                                   t3.ride_status,
                                   t4.partner_id,
                                   t4.partner,
                                   t0.from_time_str,
                                   t0.from_timezone_str,
                                   t0.to_time_str,
                                   t0.to_timezone_str,
                                   t5.start_place_id,
                                   t5.start_place,
                                   t5.longt    AS start_longt,
                                   t5.latit    AS start_latit,
                                   t6.end_place_id,
                                   t6.end_place,
                                   t6.longt    AS end_longt,
                                   t6.latit    AS end_latit,
                                   t0.distance,
                                   t0.duration,
                                   t7.vehicle_class_id,
                                   t7.vehicle_class,
                                   t0.passenger_count,
                                   t0.luggage_count,
                                   t0.trip_count,
                                   t0.children_count,
                                   t0.infant_count,
                                   t1.amount   as dispatch_amount,
                                   t1.currency as dispatch_currency,

                            FROM `elife-data-warehouse-prod.ods.ride_ride` AS t0
                                     JOIN `elife-data-warehouse-prod.ods.ride_dispatch` AS t1 ON t0.id = t1.ride_id
                                     LEFT JOIN TripType AS t2 ON t0.id = t2.ride_id
                                     LEFT JOIN RideStatus AS t3 ON t0.id = t3.ride_id
                                     LEFT JOIN Partner AS t4 ON t0.id = t4.ride_id
                                     LEFT JOIN FromPlace AS t5 ON t0.id = t5.ride_id
                                     LEFT JOIN ToPlace AS t6 ON t0.id = t6.ride_id
                                     LEFT JOIN VehicleClass AS t7 ON t0.id = t7.ride_id
                            WHERE from_time_str > '2024-01-01'
        )

        SELECT
            t.ride_id,
            t.dispatch_id,
            t.trip_type,
            t.trip_no,
            t.ride_status,
            t.partner,
            t.from_time_str,
            t.from_timezone_str,
            t.to_time_str,
            t.to_timezone_str,
            t.start_place,
            t.start_latit,
            t.start_longt,
            t.end_place,
            t.end_latit,
            t.end_longt,
            t.distance,
            t.duration,
            t.vehicle_class,
            t.passenger_count,
            t.luggage_count,
            t.trip_count,
            t.children_count,
            t.infant_count,
            t.dispatch_amount,
            t.dispatch_currency,

        FROM
            PricingTraining AS t
        WHERE
            from_time_str > '2024-01-01'
        LIMIT
            200
"""


In [ ]:
#| export
def get_dataframe_from_sql(
        credential: Credentials,
        query: str = QUERY_PRICING_TRAINING
) -> pd.DataFrame:
    """
    get dataframe from sql query in BigQuery dialect 
    
    :param
        credential: google.oauth2.service_account.Credentials
        query: string for sql query in BigQuery dialect
    :return: 
    """
    
    df = pandas_gbq.read_gbq(query,credentials=credential)
    
    return df


In [ ]:
# df = get_dataframe_from_sql(QUERY_PRCING_TRAINING,credential)

In [ ]:
# df

In [ ]:
from sqllineage.runner import LineageRunner
#from lineagex.lineagex import lineagex

In [ ]:
sql =   """
        WITH
        TripTypeID AS (
            SELECT
                t1.ride_id,
                CASE
                    WHEN t2.trip_no = t1.trip_no THEN t2.trip_type
                    ELSE NULL
                END AS trip_type_id,
            FROM
                `elife-data-warehouse-prod.ods.ride_dispatch` AS t1
                JOIN `elife-data-warehouse-prod.ods.ride_trip` AS t2 ON t1.ride_id = t2.ride_id
        ),
        TripType AS (
            SELECT
                t1.ride_id,
                t1.trip_type_id,
                t2.name AS trip_type,
            FROM
                TripTypeID AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_enum AS` t2 ON t1.trip_type_id = t2.id
        ),
        RideStatus AS (
            SELECT
                t1.id AS ride_id,
                t1.stat AS ride_status_id,
                t2.name AS ride_status,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_enum` AS t2 ON t1.stat = t2.id
        ),
        PartnerID AS (
            SELECT
                t1.id AS ride_id,
                CASE
                    WHEN t2.id = t1.partner_tran_id THEN t2.partner_id
                    ELSE NULL
                END AS partner_id,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_partner_tran` AS t2 ON t1.partner_tran_id = t2.id
        ),
        Partner AS (
            SELECT
                t1.ride_id,
                t1.partner_id,
                t2.name AS partner,
            FROM
                PartnerID AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_partner` AS t2 ON t1.partner_id = t2.id
        ),
        FromPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.from_place_id as start_place_id,
                t2.name AS start_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.dim.dim_place` AS t2 ON t1.from_place_id = t2.id
        ),
        ToPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.to_place_id as end_place_id,
                t2.name AS end_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.dim.dim_place` AS t2 ON t1.to_place_id = t2.id
        ),
        VehicleClass AS (
            SELECT
                t1.id AS ride_id,
                t1.vehicle_class_id as vehicle_class_id,
                t2.name AS vehicle_class,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_vehicle_class` AS t2 ON t1.vehicle_class_id = t2.id
        ),
        PricingTraining AS (SELECT t0.id       AS ride_id,
                                   t1.id       AS dispatch_id,
                                   t2.trip_type_id,
                                   t2.trip_type,
                                   t1.trip_no,
                                   t3.ride_status_id,
                                   t3.ride_status,
                                   t4.partner_id,
                                   t4.partner,
                                   t0.from_time_str,
                                   t0.from_timezone_str,
                                   t0.to_time_str,
                                   t0.to_timezone_str,
                                   t5.start_place_id,
                                   t5.start_place,
                                   t5.longt    AS start_longt,
                                   t5.latit    AS start_latit,
                                   t6.end_place_id,
                                   t6.end_place,
                                   t6.longt    AS end_longt,
                                   t6.latit    AS end_latit,
                                   t0.distance,
                                   t0.duration,
                                   t7.vehicle_class_id,
                                   t7.vehicle_class,
                                   t0.passenger_count,
                                   t0.luggage_count,
                                   t0.trip_count,
                                   t0.children_count,
                                   t0.infant_count,
                                   t1.amount   as dispatch_amount,
                                   t1.currency as dispatch_currency,

                            FROM `elife-data-warehouse-prod.ods.ride_ride` AS t0
                                     JOIN `elife-data-warehouse-prod.ods.ride_dispatch` AS t1 ON t0.id = t1.ride_id
                                     LEFT JOIN TripType AS t2 ON t0.id = t2.ride_id
                                     LEFT JOIN RideStatus AS t3 ON t0.id = t3.ride_id
                                     LEFT JOIN Partner AS t4 ON t0.id = t4.ride_id
                                     LEFT JOIN FromPlace AS t5 ON t0.id = t5.ride_id
                                     LEFT JOIN ToPlace AS t6 ON t0.id = t6.ride_id
                                     LEFT JOIN VehicleClass AS t7 ON t0.id = t7.ride_id
                            WHERE from_time_str > '2024-01-01'
        )

        SELECT
            t.ride_id,
            t.dispatch_id,
            t.trip_type,
            t.trip_no,
            t.ride_status,
            t.partner,
            t.from_time_str,
            t.from_timezone_str,
            t.to_time_str,
            t.to_timezone_str,
            t.start_place,
            t.start_latit,
            t.start_longt,
            t.end_place,
            t.end_latit,
            t.end_longt,
            t.distance,
            t.duration,
            t.vehicle_class,
            t.passenger_count,
            t.luggage_count,
            t.trip_count,
            t.children_count,
            t.infant_count,
            t.dispatch_amount,
            t.dispatch_currency,

        FROM
            PricingTraining AS t
        WHERE
            from_time_str > '2024-01-01'
        LIMIT
            200
        """ 

In [ ]:
# result = LineageRunner("../data/pricing_training_bigquery.sql", dialect="bigquery")
result = LineageRunner(sql, dialect="bigquery")
#from lineagex.example import example
#example("dependency_example")
#lineagex(sql="../data/pricing_training_bigquery.sql", target_schema="schema1", search_path_schema="schema1, public")

In [ ]:
print(result)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()